In [1]:
import os
import cv2
import tensorflow as tf
from keras import layers,Model
from keras.layers import BatchNormalization,Flatten,PReLU,Conv2D,UpSampling2D,LeakyReLU,Dense,Input,add
from keras.models import Model
from keras.applications import VGG19
from sklearn.model_selection import train_test_split
import numpy as np# add comments at proper places

2025-01-07 19:06:59.294491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 19:07:01.073922: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 19:07:01.577870: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 19:07:05.213730: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-07 19:07:16.999752: W tensorflow/compiler/tf2

In [3]:
 def resuidal_block(ip):

 # creating the resuidal block     resuidalModel = Conv2D(64,(3,3),padding = "same")(ip)
    resuidalModel = BatchNormalization(momentum = 0.5)(resuidalModel)
    resuidalModel = PReLU(shared_axes=[1,2])(resuidalModel)
    resuidalModel = Conv2D(64,(3,3),padding="same")(resuidalModel)
    resuidalModel = BatchNormalization(momentum = 0.5)(resuidalModel)

    return add([ip,resuidalModel])

In [4]:
def upscale_block(ip):
    upscaleModel = Conv2D(64,(3,3),padding="same")(ip)
    upscaleModel = UpSampling2D(size =2)(upscaleModel)
    upscaleModel = PReLU(shared_axes=[1,2])(upscaleModel)
    return upscaleModel

In [5]:
def createGenerator(generator_ip, numOfResBlocks):
    layers = Conv2D(64,(3,3),padding="same")(generator_ip)
    layers = PReLU(shared_axes = [1,2])(layers)
    temp = layers
    for i in range(numOfResBlocks):
        layers = resuidal_block(layers)

    layers = Conv2D(64,(3,3),padding = "same")(layers)
    layers = BatchNormalization(momentum = 0.5)(layers)
    layers = add([layers,temp])
    layers = upscale_block(layers)
    layers = upscale_block(layers)

    op = Conv2D(3,(9,9),padding = "same")(layers)
    return Model(inputs = generator_ip, outputs = op)

In [6]:
def discriminatorBlock(ip,filters,strides = 1,batchnorm =True):

    discBlock = Conv2D(filters,(3,3),strides = strides,padding="same")(ip)
    if batchnorm:
        discBlock = BatchNormalization(momentum = 0.8)(discBlock)
    discBlock = LeakyReLU(alpha = 0.2)(discBlock)
    return discBlock

In [7]:
def createDiscriminator(discriminatorIp):

    df = 64

    d1 = discriminatorBlock(discriminatorIp,df,batchnorm =False)
    d2 = discriminatorBlock(d1,df,strides =2)
    d3 = discriminatorBlock(d2,df * 2)
    d4 = discriminatorBlock(d3,df * 2,strides = 2)
    d5 = discriminatorBlock(d4,df * 4)
    d6 = discriminatorBlock(d5,df * 4,strides = 2)
    d7 = discriminatorBlock(d6,df * 8)
    d8 = discriminatorBlock(d7,df * 8,strides = 2)

    d8_5 = Flatten()(d8)
    d9 = Dense(df*16)(d8_5)
    d10 = LeakyReLU(alpha = 0.2)(d9)
    validity = Dense(1,activation = 'sigmoid')(d10)

    return Model(inputs = discriminatorIp , outputs = validity)

In [8]:
def buildVGG19(highResShape):
    vgg = VGG19(weights = "imagenet",include_top = False,input_shape = highResShape)
    return Model(inputs = vgg.inputs,outputs = vgg.layers[10].output)

In [9]:
def createCombine(genratorModel,discriminatorModel,vgg,lr_ip,hr_ip):

    gen_img = genratorModel(lr_ip)
    gen_features = vgg(gen_img)
    discriminatorModel.trainiable = False
    validity = discriminatorModel(gen_img)

    return Model(inputs = [lr_ip,hr_ip],outputs = [validity,gen_features])

In [10]:
lr_ip = Input(shape = (32,32,3))
hr_ip = Input( shape = (128,128,3))

In [11]:
generator = createGenerator(lr_ip,numOfResBlocks = 16)
generator.summary()

I0000 00:00:1736257084.189629  103988 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1736257089.179882  103988 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1736257089.180166  103988 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1736257089.192068  103988 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1736257089.192183  103988 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ p_re_lu (PReLU)     │ (None, 32, 32,    │         64 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │     36,928 │ p_re_lu[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ p_re_lu_1 (PReLU)   │ (None, 32, 32,    │         64 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │     36,928 │ p_re_lu_1[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 32, 32,    │          0 │ p_re_lu[0][0],    │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 32, 32,    │     36,928 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ p_re_lu_2 (PReLU)   │ (None, 32, 32,    │         64 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │     36,928 │ p_re_lu_2[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 32, 32,    │          0 │ add[0][0],        │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │     36,928 │ add_1[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_5[0][0]  

 Total params: 1,319,491 (5.03 MB)

 Trainable params: 1,315,267 (5.02 MB)

 Non-trainable params: 4,224 (16.50 KB)

In [13]:
discriminator = createDiscriminator(hr_ip)
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
discriminator.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_45 (Conv2D)              │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_10 (LeakyReLU)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_11 (LeakyReLU)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_12 (LeakyReLU)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_49 (Conv2D)              │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_13 (LeakyReLU)      │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_50 (Conv2D)              │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_14 (LeakyReLU)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_51 (Conv2D)              │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 16, 16, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_15 (LeakyReLU)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_52 (Conv2D)              │ (None, 8, 8, 512)      │     2,359,80

 Total params: 38,249,281 (145.91 MB)

 Trainable params: 38,245,569 (145.90 MB)

 Non-trainable params: 3,712 (14.50 KB)

In [14]:
vgg = buildVGG19((128,128,3))
print(vgg.summary())
vgg.trainable = False

gan_model = createCombine(generator, discriminator, vgg, lr_ip, hr_ip)
gan_model.compile(loss=["binary_crossentropy", "mse"], loss_weights=[1e-3, 1], optimizer="adam")
gan_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,325,568 (8.87 MB)

 Trainable params: 2,325,568 (8.87 MB)

 Non-trainable params: 0 (0.00 B)

...

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 128, 128,  │  1,319,491 │ input_layer[0][0] │
│ (Functional)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 1)         │ 38,249,281 │ functional[0][0]  │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_3        │ (None, 32, 32,    │  2,325,568 │ functional[0][0]  │
│ (Functional)        │ 256)              │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 41,894,340 (159.81 MB)

 Trainable params: 39,560,836 (150.91 MB)

 Non-trainable params: 2,333,504 (8.90 MB)

In [15]:
def train(epochs, batchSize=1, saveInterval=50):
    
    def createLowandHighResImages(directory):
        # Read the image file
        img = tf.io.read_file(directory)
        
        # Decode the image
        img = tf.image.decode_jpeg(img, channels=3)
        
        # Resize to low-resolution (32x32)
        lr_img = tf.image.resize(img, [32, 32])
        
        # Resize to high-resolution (128x128)
        hr_img = tf.image.resize(img, [128, 128])
        
        # Normalize pixel values to the range [0, 1]
        lr_img = tf.cast(lr_img, tf.float32) / 255.0
        hr_img = tf.cast(hr_img, tf.float32) / 255.0
        
        return lr_img, hr_img
    
    # Directory containing image files
    image_dir = "hr_images/*.jpg"  # Replace with your actual directory path
    
    # Create a tf.data.Dataset of file paths
    ds_train = tf.data.Dataset.list_files(image_dir)
    
    # Map the image processing function to the dataset
    ds_train = ds_train.map(createLowandHighResImages, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Optimize the dataset pipeline
    ds_train = ds_train.cache()  # Cache the dataset to speed up subsequent epochs
    ds_train = ds_train.batch(batchSize)  # Set the batch size
    ds_train = ds_train.prefetch(tf.data.AUTOTUNE)  # Prefetch the data for faster loading

    for epoch in range(epochs):
        g_losses = []
        d_losses = []
        
        for lr_imgs, hr_imgs in ds_train:
            # Prepare labels
            fakeLabel = np.zeros((batchSize, 1))
            realLabel = np.ones((batchSize, 1))  # Should be 1 for real data
            
            # Generate fake images
            fake_imgs = generator.predict_on_batch(lr_imgs)
            
            # Train the discriminator
            discriminator.trainable = True
            d_loss_gen = discriminator.train_on_batch(fake_imgs, fakeLabel)
            d_loss_real = discriminator.train_on_batch(hr_imgs, realLabel)
            discriminator.trainable = False
            
            d_loss = tf.reduce_mean([d_loss_real, d_loss_gen])
            
            # Prepare VGG features
            hr_imgs_preprocessed = tf.keras.applications.vgg19.preprocess_input(hr_imgs * 255.0)  # Scale back to [0, 255] for VGG preprocessing
            image_features = vgg.predict(hr_imgs_preprocessed)
            
            # Train the GAN model
            g_loss = gan_model.train_on_batch([lr_imgs, hr_imgs], [realLabel, image_features])
            
            # Track losses
            d_losses.append(d_loss)
            g_losses.append(g_loss)
        
        # Calculate the average losses for generator and discriminator
        g_loss = np.mean(g_losses, axis=0)
        d_loss = np.mean(d_losses, axis=0)
        
        # Report progress
        print(f"Epoch: {epoch+1}, Generator Loss: {g_loss}, Discriminator Loss: {d_loss}")
    
        # Save the generator model periodically
        if (epoch + 1) % saveInterval == 0:
            generator_save_path = f"generator_epoch_{epoch+1}"  # Save as a directory
            generator.save(generator_save_path)  # Saves in TensorFlow format


In [ ]:
train(epochs=1, batchSize=1, saveInterval=1)

I0000 00:00:1736257155.813687  104911 service.cc:146] XLA service 0x7f32340098d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736257155.813901  104911 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-01-07 19:09:16.080259: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-07 19:09:16.838436: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1736257161.560220  104911 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-01-07 19:09:31.033563: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but thi

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 807ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━